# Ngày 6: SPARK SQL - DATAFRAME

### 1. Giới thiệu về Spark SQL

1.1. Định Nghĩa

- Spark SQL là module xử lý dữ liệu có cấu trúc trong Apache Spark
- Cung cấp giao diện để làm việc với dữ liệu có cấu trúc và bán cấu trúc

1.2 Ưu Điểm

- Tích hợp truy vấn SQL với xử lý dữ liệu phân tán
- Hiệu suất cao nhờ Catalyst Optimizer
- Hỗ trợ nhiều nguồn dữ liệu

### 2. Kiến Trúc Spark SQL

2.1 Thành Phần Chính

- DataFrame API
- Dataset API
- SQL Interface

2.2 Catalyst Optimizer\
Quá trình tối ưu hóa truy vấn
Các giai đoạn:
- Phân tích cú pháp
- Kiểm tra logic
- Tối ưu hóa kế hoạch
- Sinh mã thực thi

2.3 Tungsten Execution Engine

- Tối ưu hóa bộ nhớ và xử lý
- Quản lý bộ nhớ hiệu quả
- Giảm overhead GC

### 3. DataFrame và Dataset

3.1 So Sánh
- RDD: Không có cấu trúc, hiệu suất thấp
- DataFrame: Có cấu trúc, tối ưu hơn
- Dataset: Kết hợp ưu điểm RDD và DataFrame
  
3.2 Các Phép Biến Đổi
- Transformation: select, filter, groupBy
- Action: show, count, collect

### 4. Tạo DataFrame

4.1 Từ RDD
```python
# Chuyển RDD sang DataFrame
rdd = sc.parallelize([(1, "Alice"), (2, "Bob")])
df = spark.createDataFrame(rdd, ["id", "name"])
```

4.2 Từ File

```python
# CSV
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("data.csv")

# JSON
df_json = spark.read.json("data.json")
```

4.3 Từ JDBC

```python
df_jdbc = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/db") \
    .option("dbtable", "users") \
    .option("user", "username") \
    .option("password", "password") \
    .load()
```

### 5. Truy vấn SQL

5.1 Đăng kí view

**Tạo view tạm thời**
df.createOrReplaceTempView("social_posts")

**Truy vấn SQL**
```python
result = spark.sql("""
    SELECT 
        date, 
        COUNT(*) as post_count,
        AVG(likes) as avg_likes
    FROM social_posts
    GROUP BY date
""")
```

5.2 Các hàm thông dụng

- Aggregate: sum(), avg(), count()
- Window Functions
- Joins
- Subqueries

## Bài Tập Thực Hành

Bài 1: Tạo DataFrame từ một file CSV chứa dữ liệu mạng xã hội và thực hiện các truy vấn SQL đơn giản (ví dụ: đếm số bài đăng theo ngày, tìm các hashtag phổ biến).